# Python Tutorial 05: Example Workflow

This notebook demonstrates a typical workflow for data analysis using some of the concepts we have learned so far. First we will read data from the [Bedford Basin Monitoring Program](https://www.bio.gc.ca/science/monitoring-monitorage/bbmp-pobb/bbmp-pobb-en.php) and then we will plot the data. If you spot any mistakes or issues, please report them to christoph.renkl@dal.ca.

## Preamble
At the beginning of each notebook, you should import all the packages you will need and I also like to specify the paths to data and output directories.

In [ ]:
# import packages
from pathlib import Path
import cmocean.cm as cmo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

# path to DISP directory
dispdir = Path("/home/chrenkl/Projects/DISP/python_tutorial")

# specify the path to the directory with the raw data
raw_dir = dispdir / "data" / "raw"

# specify the path to the directory with processed data and create it if it does not exist
processed_dir = dispdir / "data" / "processed"
processed_dir.mkdir(parents=True, exist_ok=True)

# we also specify the directory where we want to save our plots
figdir = dispdir / "figures"
figdir.mkdir(parents=True, exist_ok=True)

## Define Functions
We define functions that will make the analysis easier and your code cleaner.

The first function reads the orginal CSV data file and converts it to a NetCDF file. Make sure you document everything and describe what each function does!

In [ ]:
def read_bbmp_data(datadir, outfile):
    """
    Reads Bedford Basin Monitoring Program CSV file and saves it as NetCDF file.
    
    Input:
    - outfile: name of output data file
    
    Output:
    - ds: xarray.Dataset() with data
    """
    
    # full path to data file
    fname = datadir / "bbmp_aggregated_profiles.csv"
    
    # read file into pandas DataFrame, only use certain columns
    df = pd.read_csv(fname,
                     usecols=[0, 6, 7, 13, 15])
        
    # convert time_string column to datetime format
    df["time_string"] = pd.to_datetime(df["time_string"],
                                       format="%Y-%m-%d %H:%M:%S")
    
    # set time and pressure columns as index
    df = df.set_index(["time_string", "pressure"])
    
    # there are duplicate indices, we only keep the first occurence
    df = df.loc[~df.index.duplicated(keep="first")]
    
    # convert DataFrame to xarray Dataset and rename `time_string` to `time`
    ds = df.to_xarray().rename({"time_string": "time"})
    
    # only keep measurements every half meter
    ds = ds.where((ds["pressure"] % .5 == 0.) &
                  (ds["pressure"] <= 70.), drop=True)
    
    # write Dataset tp output file
    ds.to_netcdf(outfile)
        
    return ds

The second function will create a [Hovmöller diagram](https://en.wikipedia.org/wiki/Hovm%C3%B6ller_diagram) of the data.

In [ ]:
def plot_hovmoeller(ds, vname, ax):
    """
    Plot a Hovmoeller diagram (variable as function of depth and time)
    
    Input:
    - ds: xarray.Dataset with data
    - vname: name of variable to plot
    - ax: axis handle of figure
    
    Output:
    - pcm: handle to pcolormesh plot
    """

    # The following if statement is the same as we have seen in Tutorial 03.
    
    # set parameters if variable is temperature
    if vname == "temperature":
        units = r"[$^\circ$C]" # units of temperature are degree Celsius
        name = "Temperature" 
        cmap = cmo.thermal     # we choose the `thermal` colormap of the `cmocean` package
        vmin = -2.             # the minimum value we want to show
        vmax = 20.             # the maximum value we want to show
        nlevels = 23           # number of contour levels

    # set parameters if variable is salinity
    elif vname == "salinity":
        units = "[-]"          # salinity has no units
        name = "Salinity"
        cmap = cmo.haline      # we choose the `haline` colormap of the `cmocean` package
        vmin = 28.
        vmax = 32.
        nlevels = 33           # number of contour levels

    # set parameters if variable is sigmaTheta (potential density anomaly of sea water)
    elif vname == "sigmaTheta":
        units = r"[kg m$^{-3}$]"  # the units of of density are kg/m^3
        name = r"Potential Density Anomaly $\sigma_{{\theta}}$"
        cmap = cmo.dense          # we choose the `dense` colormap of the `cmocean` package
        vmin = 21.
        vmax = 25.5
        nlevels = 37           # number of contour levels

    # set a default
    else:
        units = ''
        name = vname.capitalize()
        cmap = "viridis"
        vmin = ds[vname].min()
        vmax = ds[vname].max()
        nlevels = 31           # number of contour levels

        
    # Create title string
    title = f"Hovmoeller Diagram - {name} {units}"

    # Plot the variable as a function of time (x-axis) and pressure (y-axis)
    cs = ax.contourf(
        ds["time"].values,                       # values on x-axis
        ds["pressure"],                          # values on y-axis
        ds[vname].transpose(),                   # variable we want to plot   
        levels=np.linspace(vmin, vmax, nlevels), # specify contour levels
        cmap=cmap                                # colormap
    )  

    # set label and title
    ax.set_ylabel('Pressure [dbar]')
    ax.set_title(title)
    
    # we return the handle to the plotted object
    return cs

## Data Analysis

Typically, data analyis consists of the following steps:

1. Read dataset(s)
1. Subset and clean up data
1. Analyze data, i.e., compute trends or correlations, etc.
1. Plot results

Sometimes, the steps above become so extensive that it is worthwile to break them up into individual notebooks and save temporary results.

### Read Dataset

If we have already converted the raw data from CSV to NetCDF before, we use the file right away. Otherwise, we use the function define above to convert the raw data file.

In [ ]:
# specify the full path and name of the processed output data file
fname = processed_dir / "bbmp_aggregated_profiles.nc"

# check if data file already exists. If not download the data.
if fname.exists():
    
    print(f"The file {fname} already exists - read file...")

    # open existing data file as xarray.Dataset()
    ds =  xr.open_dataset(fname)

else:

    print(f"The file {fname} doesn't exist - convert raw CSV data file to NetCDF...")

    # download data, save file, and return xarray.Dataset()
    ds = read_bbmp_data(raw_dir, fname)

ds

### Plotting

The idea is to plot a Hovmoeller diagram (variable as function of depth and time) for all data variables in `ds`. 

In [ ]:
# get names of all data variables
dvars = list(ds.data_vars)
dvars    

In [ ]:
# Create a figure with as many axes as variables in ds
fig, axs = plt.subplots(nrows=len(dvars), ncols=1,
                        figsize=(7.5, 9.3),
                        sharex=True, sharey=True)

# plot Hovmoeller diagrams
for ii, vname in enumerate(dvars):
    pcm = plot_hovmoeller(ds, vname, axs[ii])

    # colorbar
    fig.colorbar(pcm, ax=axs[ii])

# invert y-axes
axs[ii].invert_yaxis()


# make sure the plot takes up all space on the figure
fig.tight_layout()

# save figure as a PDF
fig.savefig(figdir / f"bbmp_hovmeoller_diagrams.pdf")